<h1> Ranking </h1>
Task: create dataset based on the unique pairs of 11 movies from the imdb dataset, annotate each row with preference vote, compare the result of Bradley–Terry model output with the original dataset<br><br>

<h2>My datasets</h2>
    I have collected 11 movies (title, global ranking, relative ranking) and generated the dataset with 55 entries (left, right, label)  <br>
    <hr>
    <h3>Pair dataset</h3>
    <p style="background-color: rgb(255, 127, 0);color:black;font-weight:bold">left and right columns consist of movie titles  <span style="position:absolute;right:10em">string</span> <br></p>
    <p style="background-color: rgb(0, 255, 127);color:black;font-weight:bold">Label consist of title that is more preferred compared to other title in a pair set<span style="position:absolute;right:10em">number</span> <br></p>
    <hr>
    <h3>Movies dataset</h3>
    <p style="background-color: rgb(255, 127, 0);color:black;font-weight:bold">movie title <span style="position:absolute;right:10em">string</span> <br></p>
    <p style="background-color: rgb(0, 255, 127);color:black;font-weight:bold">movie imdb rank<span style="position:absolute;right:10em">number</span> <br></p>
    <p style="background-color: rgb(0, 255, 127);color:black;font-weight:bold">movie relative rank 1-11<span style="position:absolute;right:10em">number</span> <br></p>

<h2>Add libraries</h2>

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay, 
                             classification_report, f1_score)
import re
import stanza
from tqdm import tqdm
from crowdkit.aggregation import BradleyTerry, NoisyBradleyTerry

<h1>Read Datasets</h1>

In [93]:
df_pairs = pd.read_excel('pairs.xlsx', index_col=0)
df_movies = pd.read_excel('movies.xlsx', index_col=0)

In [94]:
df_pairs.head()

,left,right,label
0,The Dark Knight,Inception,Inception
1,The Dark Knight,Spider man across the spider verse,The Dark Knight
2,The Dark Knight,The Silence of the Lambs,The Dark Knight
3,The Dark Knight,Sen to Chihiro no kamikakushi,The Dark Knight
4,The Dark Knight,Intouchables,The Dark Knight


In [95]:
df_movies.head()

,movie,g_rank,r_rank
0,The Dark Knight,3,1
1,Inception,14,2
2,Spider man across the spider verse,19,3
3,The Silence of the Lambs,24,4
4,Sen to Chihiro no kamikakushi,32,5


In [96]:
df_movies.iloc[1]

movie     Inception
g_rank           14
r_rank            2
Name: 1, dtype: object

<h2>Fit the model</h2>

In [97]:
def createBradleyTerryDF(df):
    agg_bt = BradleyTerry(n_iter=11).fit_predict(df_pairs)
    np.random.seed(0)
    df_agg = pd.DataFrame({'bt': agg_bt}).reset_index()
    df_agg.rename(columns={'index': 'movie'}, inplace=True)
    df_agg['bt_rank'] = df_agg['bt'].rank(ascending=False).astype(int)
    return df_agg

<p>Some movies had same rank, therefor I have normalizad them. Example Inception and Dark Knight had rank 4, however Dark Knight has been accessed by me better than Inception, therefor the second have obtained rank 5</p>

In [99]:
df_bt = createBradleyTerryDF(df_pairs)
df_bt.loc[2,"bt_rank"] = 4
df_bt.loc[10,"bt_rank"] = 8
df_bt

,movie,bt,bt_rank
0,A Beautiful Mind,0.018496,9
1,Avengers: Infinity War,0.236316,1
2,Django Unchained,0.129714,4
3,Inception,0.075097,6
4,Intouchables,0.100748,5
5,Joker,0.197510,2
6,Sen to Chihiro no kamikakushi,0.000000,11
7,Spider man across the spider verse,0.052794,7
8,The Dark Knight,0.129714,3
9,The Silence of the Lambs,0.006818,10


<p>Well the model have returned promising results. Looks like top 3 movies for me are Avengers Infinity War, Joker, Dark Knight, which in fact is true. </p>

<h2>Join the dataframes</h2>

In [108]:
df = pd.concat([df_bt.set_index('movie'),df_movies.set_index('movie')], axis=1, join='inner').reset_index()
df

,movie,bt,bt_rank,g_rank,r_rank
0,A Beautiful Mind,0.018496,9,146,11
1,Avengers: Infinity War,0.236316,1,63,8
2,Django Unchained,0.129714,4,57,7
3,Inception,0.075097,6,14,2
4,Intouchables,0.100748,5,48,6
5,Joker,0.197510,2,80,9
6,Sen to Chihiro no kamikakushi,0.000000,11,32,5
7,Spider man across the spider verse,0.052794,7,19,3
8,The Dark Knight,0.129714,3,3,1
9,The Silence of the Lambs,0.006818,10,24,4


In [132]:
df['bt_rank'] = df['bt_rank'].astype('int')
df['g_rank'] = df['g_rank'].astype('int')
bt_rank = df['bt_rank'].to_numpy()
r_rank = df['g_rank'].to_numpy()
rho, p_val = stats.spearmanr(bt_rank, r_rank)
print(f"Spearman's rho (ρ): ", rho)

Spearman's rho (ρ):  -0.018181818181818184


<p>Looks like my ranking have almost no correlation with imdb ranking, which is predictable result. The reason is that imdb score is based on the assumptions of huge amount of people with different preferencess, while my dataset contains only my preferences. I believe it would have changed into good correlation if we have collected the dataset based on the preferences of each member of DDI class student.</p>

<h2>Calculate bootstrap confidence level</h2>

In [145]:
def calculateBootstrap(df):
    k = 1000
    df_bs = pd.DataFrame()

    for _ in tqdm(range(k), desc="Bootstrapping"):
        sampled_data = df.sample(n=len(df), replace=True)
        bt_model = BradleyTerry(n_iter=100).fit_predict(sampled_data[['left', 'right', 'label']])
        df_bs = pd.concat([df_bs, bt_model])
        
    df_bs.reset_index(inplace=True)
    df_bs.rename(columns={'index': 'movie', 0: 'bt'}, inplace=True)
    df_final = df_bs.groupby('movie').quantile([0.025, 0.975]).reset_index()
    df_final = df_final.pivot(index='movie', columns='level_1', values='bt')

    df_final.reset_index(inplace=True)
    df_final.rename(columns={0.025: '2.5_p', 0.975: '97.5_p'}, inplace=True)

    df_final['2.5_rank'] = df_final['2.5_p'].rank(ascending=False).astype(int)
    df_final['97.5_rank'] = df_final['97.5_p'].rank(ascending=False).astype(int)

    return df_final

In [146]:
df_bs = calculateBootstrap(df_pairs)
df_bs

Bootstrapping: 100%|██████████| 1000/1000 [00:02<00:00, 408.49it/s]


level_1,movie,2.5_p,97.5_p,2.5_rank,97.5_rank
0,A Beautiful Mind,0.000000,0.062738,10,9
1,Avengers: Infinity War,0.185045,0.274608,1,1
2,Django Unchained,0.056170,0.204566,3,4
3,Inception,0.019945,0.157415,6,6
4,Intouchables,0.038083,0.180025,5,5
5,Joker,0.124774,0.248825,2,2
6,Sen to Chihiro no kamikakushi,0.000000,0.000000,10,11
7,Spider man across the spider verse,0.011801,0.115631,7,8
8,The Dark Knight,0.054906,0.207335,4,3
9,The Silence of the Lambs,0.000000,0.029368,10,10


<p>Now looks like no need in downscaling of rankings. From the results above I would conclude that 97.5 percentile ranking is most suitable for me. However if I wanted to drop the least movies I watch I ould choose the 2.5 percentile. </p>